In [ ]:
pip install rdflib

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
import pandas as pd #read xslx
from rdflib import Graph, Literal, RDF, URIRef, Namespace 
from rdflib.namespace import FOAF , XSD, DC, FOAF, SKOS, RDF, RDFS
import urllib.parse 
from google.colab import files 
import io
import datetime

In [ ]:
uploaded = files.upload()

# Helper functions

In [ ]:
import hashlib
import uuid

def concept_uri(base_uri, input):
  m = hashlib.md5()
  m.update(input.encode('utf-8'))

  return URIRef(base_uri + m.hexdigest())

def addLiteral(subject, predicate, objectColumn, datatype=None):
  if pd.notna(row[objectColumn]):
    if datatype == None:
      g.add((subject, predicate, Literal(str(row[objectColumn]), lang='nl')))
    else:
      g.add((subject, predicate, Literal(str(row[objectColumn]), datatype=datatype)))

# Init graph

In [ ]:
g = Graph()

In [ ]:
#namspaces
org = Namespace('http://www.w3.org/ns/org#')
locn = Namespace('http://www.w3.org/ns/locn#')
dc_terms= Namespace('http://purl.org/dc/terms/')
schema = Namespace('http://schema.org/')
regorg = Namespace('http://www.w3.org/ns/regorg#')
person = Namespace('http://www.w3.org/ns/person#')
vcard = Namespace('http://www.w3.org/2006/vcard/ns#')
dbpedia = Namespace('https://dbpedia.org/ontology/')

organisatie = Namespace('https://data.vlaanderen.be/ns/organisatie')
persoon = Namespace('https://data.vlaanderen.be/ns/persoon#')
adres = Namespace('https://data.vlaanderen.be/ns/adres#')
generiek = Namespace('https://data.vlaanderen.be/ns/generiek#')
mandaat = Namespace('http://data.vlaanderen.be/ns/mandaat#')
besluit = Namespace('http://data.vlaanderen.be/ns/besluit#')

#lblod = Namespace('https://contacthub-dev.lblod.info/id/')
lblod = Namespace('https://contacthub-qa.lblod.info/id/')
os = Namespace('https://data.vlaanderen.be/id/concept/organisatiestatus/')
oc = Namespace('http://data.vlaanderen.be/id/concept/BestuursunitClassificationCode/')


In [ ]:
eb_ac = pd.read_excel('SP_DownloadLijst_Eredienstbesturen_AndereContacten_20210108_cleansed.xlsx', sheet_name='DATA')

In [ ]:
eb_ac.info()

## Category definition

In [ ]:
for category in eb_ac['Classification'].unique():
  category_id = concept_uri(oc, category)
  g.add((category_id, RDF.type, SKOS.Concept))
  g.add((category_id, SKOS.prefLabel, Literal(category, lang='nl')))

# Mapping

In [ ]:
for index, row in eb_ac.iterrows():

  abb_id = concept_uri(lblod + 'organisatie/', str(row['representatief orgaan']))

  g.add((abb_id, RDF.type, org.Organization))

  g.add((abb_id, org.classification, concept_uri(oc, row['Classification'])))

  contact_id = concept_uri(lblod + 'contactinfo/', str(row['representatief orgaan']))
  g.add((contact_id, RDF.type, schema.ContactPoint))
  addLiteral(contact_id, FOAF.page, 'Website', XSD.anyURI)
  addLiteral(contact_id, schema.telephone, 'Telefoon', XSD.string)
  addLiteral(contact_id, schema.faxNumber, 'Fax', XSD.string)
  
  address_id = concept_uri(lblod + 'adresvoorstelling/', str(row['representatief orgaan']))
  g.add((address_id, RDF.type, locn.Address))
  addLiteral(address_id, locn.thoroughfare, 'Straatnaam')
  addLiteral(address_id, adres['Adresvoorstelling.huisnummer'], 'Huisnummer', XSD.string)
  addLiteral(address_id, adres['Adresvoorstelling.busnummer'], 'Busnummer', XSD.string)
  addLiteral(address_id, locn.postCode, 'Postcode', XSD.string)
  addLiteral(address_id, adres.gemeenttenaam, 'Gemeentenaam')
  addLiteral(address_id, locn.adminUnitL2, 'Provincie')
  g.add((address_id, adres.land, Literal('België', lang='nl')))

  g.add((contact_id, locn.address, address_id))
  g.add((abb_id, schema.contactPoint, contact_id))

  g.add((abb_id, regorg.orgStatus, os.actief))
  
  addLiteral(abb_id, SKOS.prefLabel, 'representatief orgaan', XSD.string)

  #Bestuur
  bestuur = concept_uri(lblod + 'bestuursorgaan/', str(row['representatief orgaan']))
  g.add((bestuur, RDF.type, besluit.Bestuursorgaan))
  g.add((bestuur, besluit.bestuurt, abb_id))

  bestuur_temporary = concept_uri(lblod + 'bestuursorgaan/', (row['representatief orgaan'] + str(datetime.datetime.now())))
  g.add((bestuur_temporary, RDF.type, besluit.Bestuursorgaan))
  g.add((bestuur_temporary, generiek.isTijdspecialisatieVan, bestuur))

  # Rols
  person_id = concept_uri(lblod + 'persoon/', str(row['Gebruikte Voornaam']) + str(row['Achternaam']))
  g.add((person_id, RDF.type, person.Person))
  addLiteral(person_id, persoon.gebruikteVoornaam, 'Gebruikte Voornaam')
  addLiteral(person_id, FOAF.familyName, 'Achternaam')

  ## Mandataris
  person_mandataris = concept_uri(lblod + 'mandataris/', str(row['Gebruikte Voornaam']) + str(row['Achternaam']) + str(row['representatief orgaan']) + str(row['Rol'].lower().replace(" ", "")))
  g.add((person_mandataris, RDF.type, mandaat.Mandataris))
  g.add((person_id, mandaat.isAangesteldAls, person_mandataris))
  g.add((person_mandataris, mandaat.isBestuurlijkeAliasVan, person_id))
  #start
  #einde
  #status ~ cf loket lokale besturen PoC https://poc-form-builder.relance.s.redpencil.io/codelijsten
  person_mandaat = concept_uri(lblod + 'mandaat/', str(row['Gebruikte Voornaam']) + str(row['Achternaam']) + str(row['representatief orgaan']))
  g.add((person_mandaat, RDF.type, mandaat.Mandaat))
  rol_concept = concept_uri(lblod + 'concept/BestuursFunctieCode/', str(row['Rol'].lower().replace(" ", "")))
  g.add((person_mandaat, org.role, rol_concept))
  g.add((person_mandataris, org.holds, person_mandaat))

  g.add((bestuur_temporary, org.hasPost, person_mandaat))
  #g.add((person_mandaat, org.holds, bestuur_temporary))

In [ ]:
print(g.serialize(format='turtle').decode('UTF-8'))

In [ ]:
g.serialize('eb_ac-qa.ttl',format='turtle')